# Import And Cleanup

## 1. Import library

In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

## 2. Download the page with Toronto postal code

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
        
# Get the data form wikipedia.
html = requests.get(url)

if html.status_code == 200:
    print('Data downloaded!')
else:
    print('Error! Please check the url or your network')

Data downloaded!


## 3. Extract the table from the html respond

In [3]:
# Find the table with class "wikitable sortable" and then get the "tbody" section
tp = BeautifulSoup(html.content, 'html.parser').find('table',{'class':'wikitable sortable'}).find('tbody')

## 4. Convert the wikitable into dataframe

In [4]:
# Find all rows
rows = tp.findAll('tr')

toronto_postal = []

# Go through each row, append text of each row to list. We only get column 1-3 because 0 and 4 is empty
for row in rows:
    toronto_postal.append(row.text.split('\n')[1:4])

# Convert the list to dataframe, use the first element of list as columns name
tp_df = pd.DataFrame(toronto_postal, columns = toronto_postal[0])

# Drop the fisrt row because it's a header
tp_df.drop([0],inplace=True)

# Preview the dataframe
print(tp_df.shape)
tp_df.head()

(287, 3)


,Postcode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront


## 5. Cleanning

### a. Clean up row **Borough** set as Not assigned

In [5]:
# Get list of indexes for which column Borough has value Not assigned
indexNA = tp_df[tp_df['Borough'] == 'Not assigned'].index

# Delete row with Not assigned Borough
tp_df.drop(indexNA , inplace=True)

# Sort by Postcode column (optional)
#tp_df.sort_values(by='Postcode', inplace=True)

# Reset the index
tp_df.reset_index(drop=True, inplace=True)

# Preview the dataframe
print(tp_df.shape)
tp_df.head()

(210, 3)


,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor


**Let's see how many unique postcode in the data frame**

In [6]:
len(tp_df['Postcode'].unique())

103

### b. Join the neighbourhood with same postcode together

In [7]:
tp_join = tp_df.groupby(['Postcode', 'Borough'], as_index=False)['Neighbourhood'].apply(' ,'.join).reset_index()

# Set the columns name
tp_join.columns=['PostalCode', 'Borough', 'Neighbourhood']

### c. Set value for row with 'Not assigned' 'Neighbourhood' value

In [8]:
tp_join['Neighbourhood'] = np.where(tp_join['Neighbourhood'] == 'Not assigned', tp_join['Borough'], tp_join['Neighbourhood'])

# Preview the dataframe
tp_join.head(5)

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge ,Malvern"
1,M1C,Scarborough,"Highland Creek ,Rouge Hill ,Port Union"
2,M1E,Scarborough,"Guildwood ,Morningside ,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### d. How many rows in the dataframe?

In [9]:
tp_join.shape[0]

103

**So we have 103 rows**